---
title: Neural Networks (Draft)
---

In previous chapters, we covered the fundamentals of PyTorch and applied it to a linear regression example. Now, we will expand on that knowledge by constructing neural networks using PyTorch. The dataset we will be utilizing is the `MNIST png` dataset from [Kaggle](https://www.kaggle.com/datasets/jidhumohan/mnist-png), as opposed to the CSV version, for a more practical experience.

## Downloading Data from Kaggle 

Here are few steps you need to perform before we download the data - 

- If you don’t have a Kaggle account, you can make one for free [here](https://www.kaggle.com/).
- To download the dataset, you will need `kaggle` installed, you can run the following command in notebook or CLI.
    ```{python}
    !pip install kaggle >> /dev/null
    ```
- Have a `kaggle.json` stored in `~/.kaggle`. You can get your token by going to Your Profile -> Account -> Create New API Token.


Once you have the above three steps done, run the API command provided:

In [14]:
!kaggle datasets download -d jidhumohan/mnist-png -p "../data/"

 87%|█████████████████████████████████     | 51.0M/58.6M [00:01<00:00, 33.4MB/s]
100%|██████████████████████████████████████| 58.6M/58.6M [00:01<00:00, 33.9MB/s]


To examine the file system, we will utilize the [fastcore](https://fastcore.fast.ai/) `Path` function. It enhances the functionality of python's `Path` class and simplifies the process of inspecting directories and folders.

In [2]:
from fastcore.xtras import Path
zip_path = Path("../data/mnist-png.zip")
zip_path.exists() # Check if the file exist

True

The data has been persisted to the `mnist-png.zip` file on the local system, within the `../data` directory. The next step is to utilize the `zipfile` package to extract the contents of the archive.

:::{.callout-warning}
The execution of the following code block will take a significant amount of time(6-10 mins) as it involves the extraction of 70,000 PNG images.
:::

In [11]:
# Output directory
dPath = Path("../data/")

# Unzipping data file in output directory
import zipfile
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(str(dPath))

# Removing the original zip file
zip_path.unlink()

# Removing duplicate folder in the unzipped data
import shutil
dPath = dPath/'mnist_png'
shutil.rmtree(dPath/'mnist_png')

Next, we inspect the extracted folder.

In [3]:
dPath.ls()

(#2) [Path('../data/mnist_png/testing'),Path('../data/mnist_png/training')]

Data contains of two folder `training` and `testing`. Next, we inspect `training` folder.

In [4]:
(dPath/'training').ls()

(#10) [Path('../data/mnist_png/training/0'),Path('../data/mnist_png/training/1'),Path('../data/mnist_png/training/2'),Path('../data/mnist_png/training/3'),Path('../data/mnist_png/training/4'),Path('../data/mnist_png/training/5'),Path('../data/mnist_png/training/6'),Path('../data/mnist_png/training/7'),Path('../data/mnist_png/training/8'),Path('../data/mnist_png/training/9')]

The `training` folder comprises of subfolders for each digit ranging from `0` to `9`.

In [5]:
(dPath/'training/0').ls()

(#5923) [Path('../data/mnist_png/training/0/1.png'),Path('../data/mnist_png/training/0/1000.png'),Path('../data/mnist_png/training/0/10005.png'),Path('../data/mnist_png/training/0/10010.png'),Path('../data/mnist_png/training/0/10022.png'),Path('../data/mnist_png/training/0/10025.png'),Path('../data/mnist_png/training/0/10026.png'),Path('../data/mnist_png/training/0/10045.png'),Path('../data/mnist_png/training/0/10069.png'),Path('../data/mnist_png/training/0/10071.png')...]

Each of these digit subfolders contains images. We will proceed to load a few of these images.

In [6]:
from PIL import Image
from IPython.display import display
for img in [Image.open((dPath/'training/0').ls()[0]), 
            Image.open((dPath/'training/1').ls()[0]),
            Image.open((dPath/'training/2').ls()[0]),
            Image.open((dPath/'training/3').ls()[0])]: 
    display(img)

## Creating Dataset Object 

As previously discussed, prior to training the model, it is necessary to establish a data pipeline in PyTorch. This includes defining a `Dataset` object and subsequently loading it via a PyTorch `Dataloader`.

### Using Pure Pytorch

Initially, we will demonstrate the process of constructing a custom image `Dataset` object using pure PyTorch. To begin, we will import the necessary libraries.

In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset
import glob
import numpy as np

The `glob` library can be utilized to obtain the filepaths of all images within a directory.

In [8]:
train_paths = glob.glob(str(dPath/'training/**/*.png'))
test_paths = glob.glob(str(dPath/'testing/**/*.png'))
print(f'Training images count: {len(train_paths)} \nTesting images count: {len(test_paths)}')
print(train_paths[0:5])

Training images count: 60000 
Testing images count: 10000
['../data/mnist_png/training/0/1.png', '../data/mnist_png/training/0/1000.png', '../data/mnist_png/training/0/10005.png', '../data/mnist_png/training/0/10010.png', '../data/mnist_png/training/0/10022.png']


By utilizing `glob`, we have successfully obtained the filepaths of all images within the `training` and `testing` folders. We can see there are 60,000 training images and 10,000 testing images. The next step is to extract the labels from the folder names.

In [30]:
train_targets = list(map(lambda x: int(x.split('/')[-2]), train_paths))
test_targets  = list(map(lambda x: int(x.split('/')[-2]), test_paths))
print(f'Training labels count: {len(train_targets)} \nTesting labels count: {len(test_targets)}')
print(np.random.choice(np.array(train_targets),5))

Training labels count: 60000 
Testing labels count: 10000
[2 3 8 5 7]


Now let's define our custom image `Dataset` class.

In [10]:
class ImageDataset(Dataset):
    def __init__(self, X, y):
        self.img_paths = X
        self.targets  = y

    def __len__(self): 
        return len(self.img_paths)

    def __getitem__(self, idx):
        current_sample = torch.tensor(np.array(Image.open(self.img_paths[idx]))).flatten()/255.
        current_target = self.targets[idx]
        return (
            current_sample, 
            current_target
        )

As we can see above, `ImageDataset` is a custom PyTorch `Dataset` class. Let's walk through the components - 

- The class takes two inputs in its constructor, `X` and `y`, which are lists of image file paths and corresponding labels respectively. These are stored as class variables `self.img_paths` and `self.targets`.
- The `__len__` method returns the number of images in the dataset by returning the length of `self.img_paths` list.
- The `__getitem__` method is called when a specific sample is requested from the dataset. It takes an index as an argument, and returns a tuple of the image data and the corresponding label for that index. The image is processed as follows - 
    - It opens the image file at the index passed in the argument using PIL(Python Imaging Library) `Image.open` function
    - Converts it to a numpy array
    - Flattens it (convert it from 28x28 2d array to 784 1-d array)
    - Normalizes it by dividing by 255 floating number 
    

We will now proceed to instantiate our `ImageDataset` class for both the`training` and `testing` datasets

In [12]:
train_ds = ImageDataset(X=train_paths, y=train_targets)
test_ds = ImageDataset(X=test_paths, y=test_targets)

print(f'One object: Image Tensor of shape {train_ds[0][0].shape}, Label: {train_ds[0][1]}')
print(f'One object: Image Tensor of shape {train_ds[20000][0].shape}, Label: {train_ds[20000][1]}')

One object: Image Tensor of shape torch.Size([784]), Label: 0
One object: Image Tensor of shape torch.Size([784]), Label: 3


### Using Torchvision

Now we have seen how to create a cusomt `ImageDataset` object. Let's see how to simplify this by using the `torchvision` package. The `torchvision` package consists of popular datasets, model architectures, and common image transformations for computer vision.

In [13]:
from torchvision import datasets
from torchvision import transforms
from tqdm.auto import tqdm

In [14]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(), 
    transforms.Lambda(lambda x: torch.flatten(x))
    ])

## Create a dataset
train_ds = datasets.ImageFolder(root = dPath/'training/', 
                                      transform=transform)

test_ds = datasets.ImageFolder(root=dPath/'testing', transform=transform)

print(f'Length of train dataset: {len(train_ds)}, test_dataset: {len(test_ds)}')
print(f'One object: Image Tensor of shape {train_ds[0][0].shape}, Label: {train_ds[0][1]}')

Length of train dataset: 60000, test_dataset: 10000
One object: Image Tensor of shape torch.Size([784]), Label: 0


## Create a Dataloader

Create a dataloader using `torch.utils.data.DataLoader` function.

In [15]:
import os
num_workers = int(os.cpu_count()/2)
train_dls = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=num_workers)
test_dls = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=num_workers)

Let's look at one batch

In [16]:
batch = next(iter(train_dls))
batch[0].shape, batch[1]

(torch.Size([128, 784]),
 tensor([5, 4, 1, 5, 7, 5, 4, 7, 2, 1, 5, 7, 6, 5, 8, 6, 3, 7, 8, 0, 4, 4, 4, 0,
         6, 7, 1, 4, 0, 6, 3, 9, 1, 0, 1, 9, 4, 1, 0, 1, 9, 3, 8, 2, 6, 2, 1, 2,
         1, 0, 2, 4, 7, 4, 7, 3, 3, 4, 3, 3, 4, 4, 7, 3, 3, 4, 6, 5, 1, 0, 2, 3,
         0, 4, 5, 7, 1, 5, 0, 1, 1, 3, 0, 0, 1, 4, 0, 6, 2, 3, 8, 1, 8, 1, 2, 5,
         5, 8, 9, 9, 9, 3, 1, 1, 3, 4, 1, 7, 8, 0, 1, 1, 2, 9, 1, 5, 3, 4, 0, 6,
         1, 4, 0, 8, 9, 1, 7, 4]))

## Defining our Training and Validation loops

In [17]:
## Training loop
def train_one_epoch(model, data_loader, optimizer, loss_func):
    total_loss, nums = 0, 0
    for batch in tqdm(iter(data_loader)):
        ## Taking one mini-batch
        xb, yb = batch[0].to(dev), batch[1].to(dev)
        y_pred = model.forward(xb)
        
        ## Calculation mean square error per min-batch
        nums += len(yb)
        loss = loss_func(y_pred, yb)
        total_loss += loss.item() * len(yb)

        ## Computing gradients per mini-batch
        loss.backward()
        
        ## Update model parameters and zero grad
        optimizer.step()
        optimizer.zero_grad()
    
    return  total_loss / nums
        
def validate_one_epoch(model, data_loader, loss_func):
    loss, nums, acc = 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(iter(data_loader)):
            xb, yb = batch[0].to(dev), batch[1].to(dev)
            y_pred = model.forward(xb)
            nums += len(yb)
            loss += loss_func(y_pred, yb).item() * len(yb)
            acc += sum(y_pred.argmax(axis=1) == yb).item()
    return loss/nums, acc/nums

## Training using a Fully Connected/ Multi Layer Perceptron Model

In [18]:
class MLP(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_in, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, n_out)
        )
    def forward(self, x):
        return self.model(x)

Let's test our untrained model accuracy.

In [19]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
loss_func = nn.CrossEntropyLoss()
model = MLP(784,10).to(dev)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 5

In [20]:
test_loss, test_acc = validate_one_epoch(model=model, data_loader=test_dls, loss_func=loss_func)
print(f"Random model: Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

  0%|          | 0/79 [00:00<?, ?it/s]

Random model: Test Loss: 2.3025, Test Accuracy: 0.0772


In [21]:
def fit(epochs, model, loss_func, opt, train_dls, valid_dls):
    for epoch in range(5):    
        train_loss = train_one_epoch(model=model, data_loader=train_dls, optimizer=optimizer, loss_func=loss_func)
        test_loss, test_acc = validate_one_epoch(model=model, data_loader=valid_dls, loss_func=loss_func)
        print(f"Epoch {epoch+1},Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Valid Accuracy: {test_acc:.4f}")

In [22]:
fit(epochs, model, loss_func, optimizer, train_dls, test_dls)

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 1,Train Loss: 2.2945, Test Loss: 2.2852, Valid Accuracy: 0.1614


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 4,Train Loss: 2.2310, Test Loss: 2.2131, Valid Accuracy: 0.4615


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 5,Train Loss: 2.1983, Test Loss: 2.1752, Valid Accuracy: 0.5695


Let's train the model now for 5 epochs and we will use `AdamW` optimizer instead of `SGD` from `torch.optim` module.

In [23]:
model = MLP(784,10).to(dev)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
fit(epochs, model, loss_func, optimizer, train_dls, test_dls)

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 1,Train Loss: 0.3480, Test Loss: 0.1660, Valid Accuracy: 0.9501


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 4,Train Loss: 0.0659, Test Loss: 0.0758, Valid Accuracy: 0.9759


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 5,Train Loss: 0.0490, Test Loss: 0.0700, Valid Accuracy: 0.9797


We can see after 5 epochs we have a fairly accurate model with 98% accuracy as compared to random prediction of 12%.

## Training using a simple CNN model

In [24]:
import torch.nn.functional as F
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

In [25]:
model = Mnist_CNN().to(dev)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
fit(epochs, model, loss_func, optimizer, train_dls, test_dls)

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 1,Train Loss: 1.8228, Test Loss: 1.4976, Valid Accuracy: 0.5442


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 4,Train Loss: 1.1286, Test Loss: 1.0886, Valid Accuracy: 0.6187


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 5,Train Loss: 1.0741, Test Loss: 1.0454, Valid Accuracy: 0.6308
